In [1]:
import pandas as pd
import os
import glob
import re
import numpy as np

In [91]:
path_to_data = '..//00_data/train_test_data'

In [92]:
X_train = pd.read_csv(os.path.join(path_to_data, 'X_train.csv'))

## Cleaning

In [99]:
def drop_mostly_empty_columns(dataframe, missing_frac_threshold, to_keep):
    frac_missing = pd.isna(dataframe).mean()
    mostly_empty_columns = frac_missing[frac_missing > missing_frac_threshold].index

    mostly_empty_columns = [col for col in mostly_empty_columns if col not in to_keep]

    print('Dropping {} columns: {}'.format(len(mostly_empty_columns), mostly_empty_columns))
    dataframe = dataframe.drop(mostly_empty_columns, axis=1)
    return dataframe

In [104]:
X_train = drop_mostly_empty_columns(X_train, 0.1, [])

Dropping 0 columns: []


In [89]:
def drop_columns_with_low_variance(dataframe, threshold):
    print('Dropped columns with variance low than {}'.format(threshold))
    dataframe.drop(dataframe.std()[dataframe.std() < threshold].index.values, axis=1, inplace = True)
    return dataframe

In [94]:
X_train = drop_columns_with_low_variance(X_train, 0.3)

Dropped columns with variance low than 0.3


In [146]:
def drop_highly_correlated_columns(dataframe, threshold):
    col_corr = set() # Set of all the names of deleted columns
    corr_matrix = dataframe.corr()
    for i in range(len(corr_matrix.columns)):
        for j in range(i):
            if (corr_matrix.iloc[i, j] >= threshold) and (corr_matrix.columns[j] not in col_corr):
                colname = corr_matrix.columns[i] # getting the name of column
                col_corr.add(colname)
                if colname in dataframe.columns:
                    del dataframe[colname] # deleting the column from the dataset
    return dataframe

In [147]:
X_train = drop_highly_correlated_columns(X_train, 0.9)

In [157]:
path_to_data = '..//00_data/track_1'
temp = pd.read_csv(os.path.join(path_to_data, 'meteo_1day.csv'))
temp.head()

,station_id,year,month,day,route_type,snow_coverage_near_station,snow_coverage_route,ice_crust_route,snow_height_aver,snow_height_max,...,temperature_320cm_qual,temperature_ks_5cm,temperature_ks_5cm_qual,temperature_ks_10cm,temperature_ks_10cm_qual,temperature_ks_15cm,temperature_ks_15cm_qual,temperature_ks_20cm,temperature_ks_20cm_qual,date
0,24538,1985,1,1,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1985-01-01
1,24538,1985,1,2,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1985-01-02
2,24538,1985,1,3,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1985-01-03
3,24538,1985,1,4,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1985-01-04
4,24538,1985,1,5,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1985-01-05


In [158]:
pd.crosstab(temp.temperature_20cm_qual, temp.temperature_20cm)

temperature_20cm,-274.0,-273.0,-271.0,-269.0,-267.0,-266.0,-265.0,-264.0,-263.0,-262.0,...,281.0,283.0,284.0,285.0,286.0,287.0,288.0,289.0,290.0,9999.0
temperature_20cm_qual,,,,,,,,,,,,,,,,,,,,,
0.0,2,1,1,1,2,3,2,6,1,4,...,3,3,2,2,2,1,1,1,0,0
1.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,2,0,1,0
2.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,10579


In [159]:
# 1, 2, 3, 4, 8 - ошибочное значение
# 5, 6, 7 - сомнительное значение
# 9 - отсутствует значение
for col in list(temp.filter(regex='qual')):
    print(temp[col].value_counts(dropna = False))

NaN    213282
0.0    111438
9.0     10579
1.0        43
3.0        35
2.0        17
Name: temperature_20cm_qual, dtype: int64
NaN    213282
0.0    107152
9.0     13134
5.0      1726
1.0        63
3.0        15
2.0        13
7.0         9
Name: temperature_40cm_qual, dtype: int64
NaN    213282
0.0    107654
9.0     14128
1.0       120
3.0       115
2.0        94
8.0         1
Name: temperature_80cm_qual, dtype: int64
NaN    213282
0.0     68257
9.0     53553
2.0       176
3.0        68
1.0        56
5.0         2
Name: temperature_120cm_qual, dtype: int64
NaN    213282
0.0    106652
9.0     14912
2.0       215
1.0       215
3.0       115
8.0         3
Name: temperature_160cm_qual, dtype: int64
NaN    213282
9.0     72521
0.0     49111
5.0       187
1.0       169
2.0        99
3.0        25
Name: temperature_240cm_qual, dtype: int64
NaN    213282
0.0     82212
9.0     38342
5.0      1190
1.0       185
2.0       112
3.0        71
Name: temperature_320cm_qual, dtype: int64
NaN    218146
9.

In [ ]:
# Значения, в которых не уверены? (Threshold)
def drop_not_reliable_features(dataframe):
    

In [ ]:
X_train.head()

In [ ]:
# One-hot-encoding for categorical values (define categorical)
X_train = pd.get_dummies(data, columns=[''], drop_first=False)

## Feature engineering

In [116]:
path_to_data = '..//00_data/track_1'

In [163]:
hydro_coord = pd.read_csv(os.path.join(path_to_data, 'hydro_coord.csv'))

In [164]:
hydro_coord

,station_id,name,lat,lon,distance_from_source,drainage_area,z_null
0,3019,КИРЕНСК - Р.ЛЕНА,57.770000,108.070000,1140.0,92200,249.38
1,3021,ЗМЕИНОВО - Р.ЛЕНА,57.780000,108.320000,1157.0,140000,245.25
2,3024,ВИЗИРНЫЙ - Р.ЛЕНА,58.880000,111.250000,1456.0,176000,194.21
3,3027,ВИТИМ (ГП ПРИ ВИТИМ АЭ) - Р.ЛЕНА,59.450000,112.583333,1575.0,199000,175.51
4,3028,ПЕЛЕДУЙ - Р.ЛЕНА,59.650000,112.800000,1601.0,425000,171.30
5,3029,КРЕСТОВСКОЕ - Р.ЛЕНА,59.730000,113.170000,1639.0,440000,166.20
6,3030,ЛЕНСК - Р.ЛЕНА,60.720000,114.880000,1786.0,450000,152.47
7,3031,НЮЯ (ГП ПРИ НЮЯ МП) - Р.ЛЕНА,60.530000,116.230000,1870.0,453000,147.21
8,3032,МАЧА (ГП ПРИ МАЧА М) - Р.ЛЕНА,59.930000,117.600000,1981.0,538000,138.90
9,3035,ОЛЕКМИНСК - Р. ЛЕНА,60.366667,120.416667,2191.0,560000,122.72


In [ ]:
hydro_coord=hydro_coord.sort_values(by = 'distance_from_source')

In [169]:
# calculate_height_difference
# distance_from_source - расстояние от истока
# z_null - высотная отметка нуля графика на гидропосте
def calculate_height_difference(dataframe_with_stations):
    dataframe_with_stations=dataframe_with_stations.sort_values(by = 'distance_from_source')
    dataframe_with_stations['height_difference'] = dataframe_with_stations['z_null'] - dataframe_with_stations['z_null'].shift(1) 
    dataframe_with_stations['distance_to_previous'] = dataframe_with_stations['distance_from_source'] - dataframe_with_stations['distance_from_source'].shift(1)
    dataframe_with_stations['height_diff_by_dist'] = dataframe_with_stations['height_difference']/dataframe_with_stations['distance_to_previous']
    dataframe_with_stations.fillna(0, inplace = True)
    return dataframe

In [ ]:
hydro_coord = calculate_height_difference(hydro_coord)

In [168]:
hydro_coord.head()

,station_id,name,lat,lon,distance_from_source,drainage_area,z_null,height_difference,distance_to_previous,height_diff_by_dist
25,3554,ЧУЯ - Р.ЧУЯ,59.26,112.41,509.9,18400,179.00,0.00,0.0,0.000000
19,3087,ШОРОХОВА - Р.КИРЕНГА,57.63,108.11,728.0,46500,254.24,75.24,218.1,0.344979
22,3180,ТОККО - Р.ЧАРА,60.00,119.88,730.0,62500,143.80,-110.44,2.0,-55.220000
0,3019,КИРЕНСК - Р.ЛЕНА,57.77,108.07,1140.0,92200,249.38,105.58,410.0,0.257512
1,3021,ЗМЕИНОВО - Р.ЛЕНА,57.78,108.32,1157.0,140000,245.25,-4.13,17.0,-0.242941


In [139]:
hydro_coord.groupby(['station_id'])['distance_from_source'].value_counts(dropna = False)

station_id  distance_from_source
3019        1140.0                  1
3021        1157.0                  1
3024        1456.0                  1
3027        1575.0                  1
3028        1601.0                  1
3029        1639.0                  1
3030        1786.0                  1
3031        1870.0                  1
3032        1981.0                  1
3035        2191.0                  1
3036        2216.0                  1
3037        2343.0                  1
3038        2409.0                  1
3041        2720.0                  1
3042        2767.0                  1
3045        2803.0                  1
3047        2837.0                  1
3048        2891.0                  1
3050        3117.0                  1
3087        728.0                   1
3106        1543.0                  1
3169        1280.0                  1
3180        730.0                   1
3229        2122.0                  1
3230        2272.0                  1
3554        509.9

In [112]:
# Print dropped
#list(X_train.loc[:,X_train.apply(pd.Series.nunique) == 1].columns)

In [31]:
X_train = X_train.loc[:,X_train.apply(pd.Series.nunique) != 1]

In [32]:
X_train.describe()

,Unnamed: 0,year,station_id,day,drainage_area,z_null,1_mean_stage_min,1_mean_stage_max,1_mean_ice_thickness,1_mean_snow_height_x,...,12_std_temperature_20cm,12_std_temperature_40cm,12_std_temperature_40cm_qual,12_std_temperature_80cm,12_std_temperature_80cm_qual,12_std_temperature_120cm,12_std_temperature_160cm,12_std_temperature_160cm_qual,12_std_temperature_240cm,12_std_temperature_320cm
count,6116.000000,6116.000000,6116.000000,6116.000000,6116.000000,6116.000000,6116.000000,6116.000000,6116.000000,6116.000000,...,6116.000000,6116.000000,6116.000000,6116.000000,6116.000000,6116.000000,6116.000000,6116.000000,6116.000000,6116.000000
mean,3057.500000,2001.122302,3047.525180,21.794964,-0.018848,0.057178,-0.054110,-0.054099,0.014485,0.006891,...,0.051831,0.073598,0.072362,-0.001914,0.000635,0.041738,-0.016915,-0.009645,0.094630,0.063792
std,1765.681455,11.235795,48.903739,12.707650,1.024864,0.990888,0.964532,0.964525,0.996678,0.888984,...,1.094583,1.273186,1.272645,0.975180,0.977450,1.070226,0.803811,0.813019,1.155316,1.073874
min,0.000000,1986.000000,3019.000000,0.000000,-1.250208,-1.391692,-3.049477,-3.049866,-1.665515,-1.407120,...,-0.519018,-0.120256,-0.119836,-0.098347,-0.104139,-0.351941,-0.092165,-0.101222,-0.298322,-0.352030
25%,1528.750000,1990.000000,3029.000000,11.000000,-0.485736,-0.828790,-0.366726,-0.366981,-0.744286,-0.686197,...,-0.519018,-0.120256,-0.119836,-0.098347,-0.104139,-0.351941,-0.092165,-0.101222,-0.298322,-0.352030
50%,3057.500000,2000.000000,3035.000000,22.000000,-0.428309,0.302352,0.052202,0.051968,-0.098230,-0.089571,...,-0.519018,-0.120256,-0.119836,-0.098347,-0.104139,-0.351941,-0.092165,-0.101222,-0.298322,-0.352030
75%,4586.250000,2010.000000,3045.000000,33.000000,0.587005,0.637459,0.518892,0.518681,0.631575,0.432476,...,0.376591,-0.111126,-0.119836,-0.095026,-0.104139,-0.351941,-0.090351,-0.101222,-0.298322,-0.352030
max,6115.000000,2019.000000,3230.000000,44.000000,2.397112,2.027005,1.870338,1.870194,4.220777,3.142151,...,5.766657,9.194273,9.181381,11.045442,10.919234,8.259717,13.299272,13.190900,5.496598,4.183021


In [36]:
# Drop duplicated columns

In [43]:
X_train['temperature_20cm_qual'].value_counts()

KeyError: 'temperature_20cm_qual'

In [45]:
temp = list(X_train.columns)

In [49]:
X_train['water_in_snow']

KeyError: 'water_in_snow'

In [50]:
meteo = pd.read_csv(os.path.join(path_to_data, 'track_1/meteo_1day'))

FileNotFoundError: [Errno 2] No such file or directory: '..//00_data/train_test_data\\track_1/meteo_1day'